<a href="https://colab.research.google.com/github/AntonioHenriqueAC/tcc/blob/master/google_colab/Number_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conectando o drive ao colab


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Baixando as bibliotecas necessárias

In [0]:
!pip3 install tensorflow-gpu==1.13.1

In [0]:
!pip3 install imageai --upgrade

# Treinar novo Modelo!

## 1. Baixando a CNN pré-treinada


In [0]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

## 2. Carregando os arquivos do colab e movendo para pasta number


In [0]:
!unzip "/content/drive/My Drive/Colab/number.zip"

## 3. Treinando o modelo

In [0]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="number")
trainer.setTrainConfig(object_names_array=["0","1", "2", "3", "4", "5", "6", "7", "8", "9"], batch_size=4, num_experiments=60, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

## 4. Avaliando o modelo gerado

In [0]:
metrics = trainer.evaluateModel(model_path="number/models", json_path="number/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

# Utilizar modelo já treinado.

## 1. Contar numero de arquivos nas pastas

In [0]:
import os

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/Corridas"))
file_count_corridas = len(dirs)
file_count_corridas = file_count_corridas -1
print("Número de corridas na pasta: ", file_count_corridas)
print("")
corrida = input("Qual corrida deseja identificar? : ")

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/Corridas/Corrida_" + str(corrida) +"/tags_images"))
file_count_tag = len(files)
print("")
print("Treinar Corrida_"+str(corrida))
print("Tags: ", str(file_count_tag))


## 2. Testando o modelo treinado

In [0]:
# "/content/drive/My Drive/Colab/models/number/detection_model-ex-053--loss-0012.296.h5"
# "/content/drive/My Drive/Colab/models/number/detection_model-ex-025--loss-0012.574.h5"
# "/content/drive/My Drive/Colab/models/number/detection_model-ex-034--loss-0012.508.h5"

modelPath = "/content/drive/My Drive/Colab/models/number/detection_model-ex-025--loss-0012.574.h5"
JsonPath = "/content/drive/My Drive/Colab/json/detection_config_number.json"

In [0]:
from imageai.Detection.Custom import CustomObjectDetection
import json

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()

for x in range(file_count_tag):

  if x in {0,1,2,3,4,5,6,7,8,9}:
    x = '00' + str(x)
  
  if(int(x) > 9 and int(x) < 100):
    x = '0' + str(x)

  detections = detector.detectObjectsFromImage(input_image="/content/drive/My Drive/Colab/Corridas/Corrida_"+str(file_count_corridas)+"/tags_images/barcode-00"+str(x)+".jpg", 
                                          output_image_path="/content/drive/My Drive/Colab/Corridas/Corrida_"+str(file_count_corridas)+"/tags_detected/digit-detected-"+str(x)+".jpg", 
                                          display_percentage_probability=False, 
                                          minimum_percentage_probability=60)
  def boxPoints(e):
    return e['box_points']

  detections.sort(key=boxPoints)

  print("Imagem: ", x)
  print("----------------------------")
  print("Número " , ": ", "Porcentagem ", "      :    ", "box_points")
  print("")

  def removeDuplicates():
      i=0
      for detection in detections:
        if( i != len(detections)-1 ):
            
            Xa = (detections[i]['box_points'][0] + detections[i]['box_points'][2])/2
            Ya = (detections[i]['box_points'][1] + detections[i]['box_points'][3])/2
            Xb = (detections[i+1]['box_points'][0] + detections[i+1]['box_points'][2])/2
            Yb = (detections[i+1]['box_points'][1] + detections[i+1]['box_points'][3])/2

            Dab = ( (Xb-Xa)**2 + (Yb - Ya)**2 )**1/2
            if(Dab < float(11)):
                if(detections[i]["percentage_probability"] < detections[i+1]["percentage_probability"]):
                    detections.pop(i)
                else:
                    detections.pop(i+1)

        i=i+1

  while(len(detections) > 14):
    removeDuplicates()

  num=len(detections)
  i=0
  soma=0
  for i in range(num):
    print(detections[i]["name"], "      : ", detections[i]["percentage_probability"], " : ", detections[i]["box_points"])
    soma = soma + detections[i]["percentage_probability"]

  # quebra o for ->> continue
  # if(len(detections)<int(10)):
  #     continue

  if(len(detections) != 0):
    media = soma/len(detections)

  print("")
  print("media: ", media)
  print("")
  print("Quantidade de números reconhecidos na tag: ", len(detections))
  print("")

  print("----------------------------")

  media = {"media": media}
  imagem = {"imagem ": x}
  detections.append(media)
  detections.append(imagem)

  

  with open('/content/drive/My Drive/Colab/Corridas/Corrida_'+str(file_count_corridas)+'/tags_json/tag-'+ str(x) +'.json', 'w', encoding='utf-8') as f:
      json.dump(detections, f, ensure_ascii=False, indent=4)